# 📘 Week1: GPT-5 챗봇 실습 (Colab)


## 진행 순서
- 환경 준비 및 기본 예제
- 멀티턴 챗봇 구조 이해
- ChatSession 헬퍼 구성
- 위젯 기반 챗봇 UI 실습
- 확장 아이디어 & 참고 자료


## 1) 환경 준비 및 단순 출력
Responses API를 이용한 챗봇 실습을 시작하기 전에 필수 환경을 설정합니다. Colab에서도 동일하게 실행할 수 있도록 설계되어 있습니다.


In [ ]:
!pip -qU install openai>=1.55.0 ipywidgets


### API 키 설정 및 Client 생성
- Colab 메뉴 `Settings → Variables`에 `OPENAI_API_KEY`를 저장했다면 자동으로 불러옵니다.
- 저장하지 않았다면 아래 셀 실행 시 직접 입력을 요구합니다.
- 챗봇 실습 전 API 키 유효성을 반드시 확인하세요.


In [ ]:
import os
from getpass import getpass

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
except Exception:
    pass

if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")


In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")


### Responses API 빠른 체험
챗봇의 기반이 되는 단발성 응답을 먼저 확인해 봅니다. 이 예시는 모델이 한 문장을 생성하는 단순 호출입니다.


In [ ]:
response = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a helpful assistant. Reply in Korean.",
    input="오늘 하루를 기분 좋게 시작할 수 있는 인사말을 한 문장으로 만들어 주세요."
)

print(response.output_text)


> 실습 팁: `model`, `instructions`, 입력 문장을 바꿔 보면서 응답의 길이와 어조가 어떻게 바뀌는지 비교해 보세요.


## 2) 멀티턴 챗봇 구조 이해
Responses API는 자동으로 대화 히스토리를 저장하지 않습니다. 따라서 애플리케이션이 직접 이전 메시지를 누적해 전달해야 멀티턴 문맥이 유지됩니다.


In [ ]:
context = [
    {
        "role": "developer",
        "content": "You are a helpful assistant. Reply in Korean and keep answers concise.",
    },
    {"role": "user", "content": "방금 읽은 책을 한 줄로 요약해 달라고 부탁하는 방법을 알려 줘."},
]

first = client.responses.create(
    model="gpt-5-mini",
    input=context,
)

print("첫 번째 응답:", first.output_text)

context.append({"role": "assistant", "content": first.output_text})
context.append({"role": "user", "content": "이 내용을 더 친근하게 바꿔 줄래?"})

second = client.responses.create(
    model="gpt-5-mini",
    input=context,
)
print("\n두 번째 응답:", second.output_text)


> 멀티턴을 사용하면 사용자 의도와 모델 응답이 모두 히스토리에 남습니다. 이를 바탕으로 더 자연스러운 대화를 만들 수 있습니다.


## 3) ChatSession 헬퍼 구성
반복되는 히스토리 관리 로직을 클래스로 묶으면 챗봇 실습이 훨씬 단순해집니다. 아래 헬퍼는 개발자 지침을 포함한 초기 히스토리를 자동으로 설정하고, 각 턴 실행 시 응답과 오류를 함께 반환합니다.
 

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple


@dataclass
class ChatSession:
    """Responses API 기반 멀티턴 챗봇 세션 관리 유틸리티."""

    client: OpenAI
    model: str = "gpt-5-mini"
    developer_message: str = (
        "You are a helpful assistant. You must answer in Korean and keep answers clear."
    )
    reasoning_effort: str = "medium"
    history: List[Dict[str, str]] = field(default_factory=list)
    last_response_id: Optional[str] = field(default=None, init=False)

    def __post_init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        """대화 히스토리를 초기 developer 메시지로 리셋합니다."""
        self.history = [
            {
                "role": "developer",
                "content": self.developer_message,
            }
        ]
        self.last_response_id = None

    def send(self, user_message: str) -> Tuple[str, Dict[str, str]]:
        """사용자 메시지를 추가하고 모델 응답 텍스트와 전체 response 객체를 반환합니다."""
        cleaned = user_message.strip()
        if not cleaned:
            raise ValueError("user_message는 비어 있을 수 없습니다.")

        self.history.append({"role": "user", "content": cleaned})

        response = self.client.responses.create(
            model=self.model,
            input=self.history,
            reasoning={"effort": self.reasoning_effort},
        )

        message_text = response.output_text
        self.history.append({"role": "assistant", "content": message_text})
        self.last_response_id = response.id
        return message_text, response

    def pretty_history(self) -> str:
        """현재 히스토리를 읽기 쉬운 문자열로 변환합니다."""
        lines = []
        for msg in self.history:
            prefix = "사용자" if msg["role"] == "user" else "assistant" if msg["role"] == "assistant" else msg["role"]
            lines.append(f"[{prefix}] {msg['content']}")
        return "\n".join(lines)


### ChatSession 사용 예시
아래 셀을 실행해 모델과 대화한 뒤 히스토리를 출력해 보세요. `reset()`을 호출하면 처음 상태로 돌아갑니다.


In [ ]:
chatbot = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a helpful teaching assistant. Reply in Korean with actionable suggestions.",
    reasoning_effort="low",
)

reply, raw_response = chatbot.send("Responses API 실습을 복습하는 가장 좋은 방법을 알려 줘.")
print("모델 응답:\n", reply)

print("\n현재 히스토리:\n", chatbot.pretty_history())


In [ ]:
raw_response.usage


> 토큰 사용량을 확인하며 비용 관리 전략을 설명해 보세요. 학생들에게 `reasoning.effort` 값을 바꿔 보도록 유도하면 비교 학습이 가능합니다.


## 4) 위젯 기반 챗봇 UI 실습
`ipywidgets`를 활용하면 코드 몇 줄로 인터랙티브한 챗봇 실습 환경을 만들 수 있습니다. 아래 UI는 텍스트 입력, 전송 버튼, 초기화 버튼을 제공하며 실행할 때마다 새 세션을 생성합니다.


In [ ]:
import ipywidgets as widgets
from IPython.display import Markdown, display

chat_ui_session = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a supportive study buddy. Reply in Korean with friendly tips.",
    reasoning_effort="medium",
)

log_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px"))
input_box = widgets.Text(
    placeholder="질문을 입력하고 Enter 또는 '전송'을 클릭하세요.",
    description="질문",
    layout=widgets.Layout(width="100%"),
)
send_button = widgets.Button(description="전송", button_style="primary")
reset_button = widgets.Button(description="초기화")
status_label = widgets.HTML(value="")


def append_message(role: str, text: str) -> None:
    with log_output:
        display(Markdown(f"**{role}**: {text}"))


def handle_send(_=None) -> None:
    text = input_box.value.strip()
    if not text:
        status_label.value = "<span style='color:#d9534f;'>메시지를 입력하세요.</span>"
        return

    status_label.value = ""
    append_message("사용자", text)
    input_box.value = ""
    send_button.disabled = True
    try:
        answer, _ = chat_ui_session.send(text)
        append_message("어시스턴트", answer)
    except Exception as exc:  # pragma: no cover - 노트북 데모용
        append_message("오류", f"{type(exc).__name__}: {exc}")
    finally:
        send_button.disabled = False


def handle_reset(_=None) -> None:
    chat_ui_session.reset()
    log_output.clear_output()
    status_label.value = "<span style='color:#5cb85c;'>세션이 초기화되었습니다.</span>"


send_button.on_click(handle_send)
reset_button.on_click(handle_reset)
input_box.on_submit(handle_send)

controls = widgets.HBox([send_button, reset_button, status_label])
ui = widgets.VBox([input_box, controls, log_output])

display(ui)


> 실습 지시: 학생들에게 개발자 메시지를 바꿔 보도록 요청해 보세요. 예를 들어 "답변은 표 형식으로 작성"과 같은 추가 지침을 주면 챗봇 동작이 어떻게 달라지는지 확인할 수 있습니다.


## 5) 챗봇 실습 확장 가이드
강의 후 학습자가 직접 챗봇을 확장해 볼 수 있도록 아래 아이디어를 참고하세요.


### 실습 목표
- Responses API를 사용한 멀티턴 대화 흐름을 이해하고 직접 구현한다.
- 개발자 지침과 사용자 입력을 조합해 챗봇의 성격을 설계한다.
- 위젯 UI를 통해 사용자 경험 측면의 개선 포인트를 탐색한다.

### 권장 실험
- `reasoning_effort`, `model`, `max_output_tokens` 값을 바꿔 응답 품질과 속도를 비교한다.
- 특정 도메인(예: 고객 상담, 스터디 가이드)에 맞는 developer 메시지를 작성한다.
- `ChatSession.pretty_history()` 결과를 기반으로 대화 로그를 CSV/JSON으로 저장해 본다.

### 확장 과제
- 추가 버튼을 만들어 최근 응답을 영어로 번역하거나 요약하도록 구현한다.
- `previous_response_id`를 활용해 가지치기형 대화 흐름을 구현해 본다.
- `raw_response.usage` 데이터를 시각화해 토큰 소비 패턴을 분석한다.

### 참고 자료
- @https://platform.openai.com/docs
- @https://platform.openai.com/docs/api-reference
